# Project 2 Model Classification
### Serena Shah, Osvaldo Salinas
## Part 1

### Loading the Data

In [1]:
import os
from pathlib import Path

Path("data/train/damage").mkdir(parents=True, exist_ok=True)
Path("data/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data/test/damage").mkdir(parents=True, exist_ok=True)
Path("data/test/no_damage").mkdir(parents=True, exist_ok=True)

In [2]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above

damage_all_paths = os.listdir('data_all_modified/damage')
no_damage_all_paths = os.listdir('data_all_modified/no_damage')

In [3]:
# split the image paths into train and test by randomly selecting 80% of the images in train and 20% in test.
import random

print("...............")
train_damage_paths = random.sample(damage_all_paths, int(len(damage_all_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in damage_all_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))
print("...............\n")
print("...............")
train_no_damage_paths = random.sample(no_damage_all_paths, int(len(no_damage_all_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in no_damage_all_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))
print("...............\n")

...............
train damage image count:  11336
test damage image count:  2834
len of overlap:  0
...............

...............
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0
...............



In [5]:
# copying of files in the train and test directories
import shutil

root_dir = 'data_all_modified'
split_root_dir = 'data'

# Copy damaged images to train and test directories
for p in train_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'damage', p), os.path.join(split_root_dir, 'train/damage', p))

for p in test_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'damage', p), os.path.join(split_root_dir, 'test/damage', p))

# Copy no damage images to train and test directories
for p in train_no_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'no_damage', p), os.path.join(split_root_dir, 'train/no_damage', p))

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'no_damage', p), os.path.join(split_root_dir, 'test/no_damage', p))

# Check counts to ensure files are copied correctly
print("Files in train/damage: ", len(os.listdir(os.path.join(split_root_dir, "train/damage"))))
print("Files in train/no_damage: ", len(os.listdir(os.path.join(split_root_dir, "train/no_damage"))))
print("Files in test/damage: ", len(os.listdir(os.path.join(split_root_dir, "test/damage"))))
print("Files in test/no_damage: ", len(os.listdir(os.path.join(split_root_dir, "test/no_damage"))))

Files in train/damage:  14170
Files in train/no_damage:  7152
Files in test/damage:  11186
Files in test/no_damage:  5617


### Data preprocessing

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Rescaling
train_data_dir = 'data/train/'

batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Name: protobuf
Version: 4.23.4
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /usr/local/lib/python3.11/site-packages
Requires: 
Required-by: tensorboard, tensorflow


2024-04-10 19:34:02.755844: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 19:34:02.833571: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 19:34:03.313368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 19:34:03.313614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 19:34:03.415494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Found 21322 files belonging to 2 classes.
Using 17058 files for training.
Using 4264 files for validation.


In [8]:
test_data_dir = 'data/test/'

batch_size = 2
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 16803 files belonging to 2 classes.


## Part 2
### ANN

In [9]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
print(os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'])

import keras 
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

# Intializing a sequential model
model_cnn = models.Sequential()

# Adding first conv layer with 64 filters and kernel size 3x3 , padding 'same' provides the output size same as the input size
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(img_height, img_width, 3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling to make it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with * neurons and activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(2, activation='softmax'))

# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

python
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)  

In [22]:
#fit the model from image generator
history_ann = model_cnn.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
534/534 [==============================] - 133s 247ms/step - loss: 0.5191 - accuracy: 0.7558 - val_loss: 0.5933 - val_accuracy: 0.7022
Epoch 2/20
534/534 [==============================] - 134s 251ms/step - loss: 0.3855 - accuracy: 0.8385 - val_loss: 0.4098 - val_accuracy: 0.8159
Epoch 3/20
534/534 [==============================] - 124s 232ms/step - loss: 0.2969 - accuracy: 0.8833 - val_loss: 0.3974 - val_accuracy: 0.8276
Epoch 4/20
534/534 [==============================] - 122s 228ms/step - loss: 0.2248 - accuracy: 0.9144 - val_loss: 0.2636 - val_accuracy: 0.8926
Epoch 5/20
534/534 [==============================] - 129s 242ms/step - loss: 0.1843 - accuracy: 0.9285 - val_loss: 0.1674 - val_accuracy: 0.9350
Epoch 6/20
534/534 [==============================] - 146s 273ms/step - loss: 0.1579 - accuracy: 0.9392 - val_loss: 0.1508 - val_accuracy: 0.9425
Epoch 7/20
534/534 [==============================] - 152s 284ms/step - loss: 0.1404 - accuracy: 0.9452 - val_loss: 0.1307 -

KeyboardInterrupt: 

In [ ]:
test_loss_ann, test_accuracy_ann = model_cnn.evaluate(test_rescale_ds, verbose=0)
test_accuracy_ann

### LeNet-5

In [15]:
from keras import layers
from keras import models
import pandas as pd

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(3, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d_6 (Avera  (None, 63, 63, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_7 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_4 (Flatten)         (None, 14400)             0         
                                                                 
 dense_12 (Dense)            (None, 120)              

In [16]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
534/534 [==============================] - 43s 78ms/step - loss: 0.5916 - accuracy: 0.7025 - val_loss: 0.5579 - val_accuracy: 0.7324
Epoch 2/20
534/534 [==============================] - 33s 61ms/step - loss: 0.4663 - accuracy: 0.8068 - val_loss: 0.6121 - val_accuracy: 0.7038
Epoch 3/20
534/534 [==============================] - 32s 59ms/step - loss: 0.4154 - accuracy: 0.8343 - val_loss: 0.5275 - val_accuracy: 0.7549
Epoch 4/20
534/534 [==============================] - 32s 60ms/step - loss: 0.3883 - accuracy: 0.8460 - val_loss: 0.5356 - val_accuracy: 0.7655
Epoch 5/20
534/534 [==============================] - 32s 59ms/step - loss: 0.3737 - accuracy: 0.8546 - val_loss: 0.4538 - val_accuracy: 0.7976
Epoch 6/20
534/534 [==============================] - 33s 61ms/step - loss: 0.3519 - accuracy: 0.8637 - val_loss: 0.3352 - val_accuracy: 0.8727
Epoch 7/20
534/534 [==============================] - 31s 57ms/step - loss: 0.3326 - accuracy: 0.8717 - val_loss: 0.3992 - val_accuracy:

The Lenet-5 model accuracy is shown below.

In [19]:
test_loss_lenet5, test_accuracy_lenet5 = model_lenet5.evaluate(test_rescale_ds, verbose=0)
test_accuracy_lenet5

0.9483425617218018

In [21]:
model_lenet5.save("lenet5.keras")

### Alt LeNet-5

In [23]:
from keras import layers
from keras import models
import pandas as pd

model_altlenet = models.Sequential()

# Layer 1: Convolutional layer with 32 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 64 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional layer with 128 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Convolutional layer with 128 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_altlenet.add(layers.Flatten())

# Adding dropout prevents overfitting
model_altlenet.add(layers.Dropout(0.2))

# Layer 5: Fully connected layer with 120 neurons
model_altlenet.add(layers.Dense(120, activation='relu'))

# Layer 6: Fully connected layer with 84 neurons
model_altlenet.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_altlenet.add(layers.Dense(3, activation='softmax'))

# Compile model
model_altlenet.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_altlenet.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 14, 14, 128)      

In [24]:
#fit the model from image generator
history_altlenet = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
534/534 [==============================] - 30s 56ms/step - loss: 0.1337 - accuracy: 0.9484 - val_loss: 0.1975 - val_accuracy: 0.9282
Epoch 2/20
534/534 [==============================] - 34s 64ms/step - loss: 0.1274 - accuracy: 0.9522 - val_loss: 0.2211 - val_accuracy: 0.9224
Epoch 3/20
534/534 [==============================] - 30s 56ms/step - loss: 0.1198 - accuracy: 0.9547 - val_loss: 0.1819 - val_accuracy: 0.9339
Epoch 4/20
534/534 [==============================] - 34s 62ms/step - loss: 0.1123 - accuracy: 0.9583 - val_loss: 0.2089 - val_accuracy: 0.9287
Epoch 5/20
534/534 [==============================] - 29s 54ms/step - loss: 0.1052 - accuracy: 0.9607 - val_loss: 0.2235 - val_accuracy: 0.9280
Epoch 6/20
534/534 [==============================] - 31s 57ms/step - loss: 0.0986 - accuracy: 0.9632 - val_loss: 0.1791 - val_accuracy: 0.9348
Epoch 7/20
534/534 [==============================] - 23s 43ms/step - loss: 0.0897 - accuracy: 0.9672 - val_loss: 0.1789 - val_accuracy:

In [26]:
test_loss_altlenet, test_accuracy_altlenet = model_altlenet.evaluate(test_rescale_ds, verbose=0)
test_accuracy_altlenet

2024-04-10 21:12:46.852030: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: data/test/damage/-93.978029_29.902963.jpeg; No such file or directory
2024-04-10 21:12:46.875985: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: data/test/damage/-93.988838_29.890141999999997.jpeg; No such file or directory
2024-04-10 21:12:46.876428: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: data/test/damage/-95.10424300000001_29.844515.jpeg; No such file or directory
2024-04-10 21:12:46.877423: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: data/test/damage/-93.973794_29.899791999999998.jpeg; No such file or directory
2024-04-10 21:12:46.886323: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: data/test

NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
data/test/damage/-93.978029_29.902963.jpeg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_test_function_160824]